In [143]:
import warnings
warnings.filterwarnings('ignore')
# import geoplot as gplt

# import geopandas as gpd
# import geoplot.crs as gcrs
import imageio
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import mapclassify as mc
import numpy as np
import laspy
import rasterio
from rasterio import mask
import folium

In [14]:
# !pip install geopandas

In [142]:
import geopandas

In [64]:
import json, ast
from os import stat
import pdal


def read_json(file_name):
  '''
  Read json file and return the string format
  '''

  try:
    file_path = f"../{file_name}"
    print("File Path : ", file_path)
    with open(file_path, 'r') as json_file:
      data = json.loads(json_file.read())
    return data

  except:
    print('File Not found')
    return None


def prepare_pipe(bound, us_state='IA_FullState'):
    data = read_json('pipeline.json')
    data['pipeline'][0]['bounds'] = bound
    data['pipeline'][0]['filename'] = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/"+us_state+"/ept.json"
    
    data['pipeline'][6]['filename'] = '../data/las/'+us_state+'.las'
    data['pipeline'][7]['filename'] = '../data/tif/'+us_state+'.tif'
    

    print("data LInk : " , data['pipeline'][0]['filename'])
    return data


def run_pipe(bound, us_state):
    print("Run pipe ...")
    result = prepare_pipe(bound, us_state)
    pipeline = pdal.Pipeline(json.dumps(result))
    pipe_result  = pipeline.execute()
    print("Fetching Completed!")

def test_json_reading():
    data = read_json('pipeline.json')
    print(data)
    


def test_prepare_pipe(bound):
    result = prepare_pipe(bound)
    print(result)

## Initial Files
bound = str(([-10425171.940, -10423171.940], [5164494.710, 5166494.710]))
state = 'IA_FullState'


## Test Things are working
# test_json_reading()
# test_prepare_pipe(bound)
    

run_pipe(bound, state)


Run pipe ...
File Path :  ../pipeline.json
data LInk :  https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json
Fetching Completed!


(filters.reprojection Error) GDAL failure (1) PROJ: Unrecognized horizontal grid format
(filters.reprojection Error) GDAL failure (1) PROJ: hgridshift: could not find required grid(s).
(filters.reprojection Error) GDAL failure (1) PROJ: pipeline: Pipeline: Bad step definition: inv (File not found or invalid)
(filters.reprojection Error) GDAL failure (1) PROJ: Unrecognized horizontal grid format
(filters.reprojection Error) GDAL failure (1) PROJ: hgridshift: could not find required grid(s).
(filters.reprojection Error) GDAL failure (1) PROJ: pipeline: Pipeline: Bad step definition: inv (File not found or invalid)
(filters.reprojection Error) GDAL failure (1) PROJ: Unrecognized horizontal grid format
(filters.reprojection Error) GDAL failure (1) PROJ: hgridshift: could not find required grid(s).
(filters.reprojection Error) GDAL failure (1) PROJ: pipeline: Pipeline: Bad step definition: inv (File not found or invalid)
(filters.reprojection Error) GDAL failure (1) PROJ: Unrecognized horiz

In [54]:
import pdal

In [32]:
import laspy

In [4]:
import geopandas as gpd

In [ ]:
## Read file with laspy the .laz file
## Generate POINT with x, y
## Elevation with Z
## Geopandas Data frame, two columns -> Elevation : Z, Geometry: POINTS
## Return the Dataframe

In [133]:
def read_laz(file_name):
    try:
        las = laspy.read(file_name)
        return las
    except FileNotFoundError:
        print("Log: File Not found")
las_path = '../data/las/IA_FullState.las'    
las_file = read_laz(las_path)

In [60]:
from shapely.geometry import box, Point, Polygon

In [134]:
## Computation Intensive
def generate_points_elevation(las_file):
    '''
    Return Points (x, y) and elevation (z)
    '''
    points = [Point(x, y) for x,y in zip(las_file.x, las_file.y)]
    elevation = np.array(las_file.z)
    
    return points, elevation


    
points, elevation = generate_points_elevation(las_file)

In [135]:
points[:10]

In [136]:
elevation[:10]

array([318.49, 318.37, 318.41, 318.15, 318.29, 318.31, 318.48, 318.17,
       318.2 , 318.42])

In [137]:
import geopandas as gpd

In [138]:
def create_geopandasdf(geometry_points, elevation):
    '''
    Return Geopandas data frame from elevation and geometic points
    '''
    geo_df = gpd.GeoDataFrame({"elevation": elevation, "geometry":geometry_points})
    geo_df.set_geometry('geometry')
    geo_df.set_crs(epsg=26915, inplace=True)

    return geo_df

df = create_geopandasdf(points, elevation)
df.head()


,elevation,geometry
0,318.49,POINT (-93.75000 41.92000)
1,318.37,POINT (-93.75000 41.92000)
2,318.41,POINT (-93.75000 41.92000)
3,318.15,POINT (-93.75000 41.92000)
4,318.29,POINT (-93.75000 41.92000)


In [139]:
# Run the run_pipe() first, then create_geopandasdf()

import pdal
import json
import laspy
import geopandas as gpd
import numpy as np
from shapely.geometry import box, Point, Polygon

class FetchAndLoad():
    '''
    params : 
        polygon : polygon of area we need to crop
        state : state to do the elevation
    
    instruction:
        Make sure to run them sequentially.
        run_pipe() : will generate the neccessary files
        create_geopandasdf() : generate data frame from the files
        get_geopandas_df() : will provide the data frame
        
    '''
    def __init__(self, polygon:Polygon, epsg:int=4326, state='IA_FullState'):
        self.polygon = polygon

        self.state = state
        self.epsg = epsg
        self.bounds = None
        self.crs_polygon = None
        
        self.pipe_path = '../pipeline.json'
        self.las_path = '../data/las/'+self.state+'.las'
        self.tif_path = '../data/tif/'+self.state+'.tif'
        self.api_path = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/"+self.state+"/ept.json"
        
        self.pipeline = None
        
        self.las_file = None
        self.points = None
        self.elevation = None
        self.geo_df = None
        
        
    def read_json(self):
      '''
      Read json file and return the string format
      '''

      try:
        file_path = self.pipe_path
        print("File Path : ", file_path)
        with open(file_path, 'r') as json_file:
          data = json.loads(json_file.read())
        return data

      except:
        print('File Not found')
        return None
    
    def get_bounds_and_ploygon(self):
        '''
        set bounderies and polygons after converting user CRS - coordinate system
        to the coordinate system we use ( 3857 ).
        '''
        polygon_df = gpd.GeoDataFrame([self.polygon], columns=['geometry'])
        
        polygon_df.set_crs(epsg=self.epsg, inplace=True)
        polygon_df['geometry'] = polygon_df['geometry'].to_crs(
            epsg=3857)
        minx, miny, maxx, maxy = polygon_df['geometry'][0].bounds
        
        polygon_input = 'POLYGON(('

        xcord, ycord = polygon_df['geometry'][0].exterior.coords.xy
        for x, y in zip(list(xcord), list(ycord)):
            polygon_input += f'{x} {y}, '
        polygon_input = polygon_input[:-2]
        polygon_input += '))'
        
        
        self.bounds = f"({[minx, maxx]},{[miny,maxy]})"
        self.crs_polygon = polygon_input
        
        
    def prepare_pipe(self):
        '''
        Populate The pipe line with boundary, and return pipeline
        '''
        self.get_bounds_and_ploygon()
        data = self.read_json()
        data['pipeline'][0]['bounds'] = self.bounds
        data['pipeline'][0]['filename'] = self.api_path
        
        data['pipeline'][1]['polygon'] = self.crs_polygon
        
        data['pipeline'][4]['out_srs'] = f'EPSG:{self.epsg}'

        data['pipeline'][7]['filename'] = self.las_path
        data['pipeline'][8]['filename'] = self.tif_path


        print("data LInk : " , data['pipeline'][0]['filename'])
        self.pipeline = data
        
    
    def run_pipe(self):
        '''
        Generate .las and .tif file from the pipeline
        '''
        print("Run pipe ...")
        result = self.prepare_pipe()
        pdal_pipe = pdal.Pipeline(json.dumps(self.pipeline))
        pdal_result = pdal_pipe.execute()
        print("Fetching Completed!")
        
        
    def read_laz(self):
        '''
        Read Generated Las file
        Return laspy read las file
        '''
        try:
            print("Reading Las File from :", self.las_path)
            las = laspy.read(self.las_path)
            self.las_file = las
            return las
        
        except FileNotFoundError:
            print("Log: File Not found")
            print("Please use the function run_pipe before this funciton")
            
    
    def generate_points_elevation(self):
        '''
        Return Points (x, y) and elevation (z)
        '''
        print("Generating Points from las File ...")
        points = [Point(x, y) for x,y in zip(self.las_file.x, self.las_file.y)]
        elevation = np.array(las_file.z)
        
        self.points, self.elevation = points, elevation
        print("Finished Generating Points!")
        
        return points, elevation
    
    
    def create_geopandasdf(self):
        '''
        Return Geopandas data frame from elevation and geometic points
        '''
        self.read_laz()
        self.generate_points_elevation()
        
        print("Making Geopandas Data Frame...")
        geopanda_df = gpd.GeoDataFrame({"elevation": self.elevation, "geometry":self.points})
        geopanda_df.set_geometry('geometry')
        geopanda_df.set_crs(epsg=4326, inplace=True)
        
        self.geo_df = geopanda_df
        print("Finished Making Geopandas Data Frame!")
        return geopanda_df
    
    
    def get_geopandas_df(self):
        return self.geo_df
        
    

In [130]:
## Initial Files
bound = str(([-10425171.940, -10423171.940], [5164494.710, 5166494.710]))
state = 'IA_FullState'

# fetcher = Lidar_Data_Fetch(PUBLIC_DATA_URL, epsg=epsg)
MINX, MINY, MAXX, MAXY = [-93.756155, 41.918015, -93.747334, 41.921429]

polygon = Polygon(((MINX, MINY), (MINX, MAXY),
                   (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))



check = FetchAndLoad(polygon)
check.run_pipe()

Run pipe ...
File Path :  ../pipeline.json
data LInk :  https://s3-us-west-2.amazonaws.com/usgs-lidar-public/IA_FullState/ept.json
Fetching Completed!


In [140]:
check.create_geopandasdf()
new_result =  check.get_geopandas_df()
new_result.head()

Reading Las File from : ../data/las/IA_FullState.las
Generating Points from las File ...
Finished Generating Points!
Making Geopandas Data Frame...
Finished Making Geopandas Data Frame!


,elevation,geometry
0,318.49,POINT (-93.75000 41.92000)
1,318.37,POINT (-93.75000 41.92000)
2,318.41,POINT (-93.75000 41.92000)
3,318.15,POINT (-93.75000 41.92000)
4,318.29,POINT (-93.75000 41.92000)


In [141]:
new_result.head()

,elevation,geometry
0,318.49,POINT (-93.75000 41.92000)
1,318.37,POINT (-93.75000 41.92000)
2,318.41,POINT (-93.75000 41.92000)
3,318.15,POINT (-93.75000 41.92000)
4,318.29,POINT (-93.75000 41.92000)
